In [9]:
import os
import sys
sys.path.insert(0, "/home/ziyizhang/Desktop/Projects/mitsuba-curve/mitsuba3/build/python")

import mitsuba as mi
import drjit as dr

import numpy as np

mi.set_variant("cuda_ad_rgb")
# mi.set_variant("scalar_rgb")

In [10]:
T = mi.ScalarTransform4f
scene_dict = {
    'type': 'scene',
    'integrator': {
        'type': 'path',
        'max_depth': 8
    },
    'sensor': {
        'type': 'perspective',
        'to_world': T.look_at(
            origin=[0, 0, 10],
            target=[0, 0, 0],
            up=[0, 1, 0]
        ),
        'sampler': {
            'type': 'independent',
            'sample_count': 64
        },
        'film': {
            'type': 'hdrfilm',
            'width' : 128,
            'height': 128,
        }
    },
    'light': {
        "type": "constant",
        'radiance': {
            'type': 'rgb',
            'value': 0.1,
        }
    },
    "checker": {
        'type': 'diffuse',
        'reflectance': {
            'type': 'checkerboard',
            'to_uv': mi.ScalarTransform4f.scale([10, 10, 1])
        }
    },
    "curve1": {
        "type": "bspline",
        "filename": "data_samples/curve2.txt",
        "to_world": T.translate([0, 0.6, 0]) @ T.rotate([1, 0, 0], 0) @ T.scale(1),
        "bsdf": {"type": "ref", "id": "checker"}
    },
    "curve2": {
        "type": "bspline",
        "filename": "data_samples/curve2.txt",
        "to_world": T.translate([0, -0.6, 0]) @ T.rotate([1, 0, 0], 180) @ T.scale(1),
        "bsdf": {"type": "ref", "id": "checker"}
    },
    "curve3": {
        "type": "bspline",
        "filename": "data_samples/curve2.txt",
        "to_world": T.translate([0, -1.8, 0]) @ T.rotate([1, 0, 0], 180) @ T.scale(1),
        "bsdf": {"type": "ref", "id": "checker"}
    }
}
scene = mi.load_dict(scene_dict)

In [11]:
aov_integrator = mi.load_dict({
    "type": "aov",
    'aovs': 'uv:uv,nn:sh_normal,dpdu:dp_du,dpdv:dp_dv',
    'my_image': {
        'type': 'path',
    }
})

image = aov_integrator.render(scene, spp=1024)
scene.sensors()[0].film().write(f"./aov.exr")

## Assert

In [12]:
ray = mi.Ray3f([0, 1, -1e-5], [0, -1, 0])
si = scene.ray_intersect(ray)

In [13]:
si

SurfaceInteraction[
  t = [0.1],
  time = [0],
  wavelengths = [],
  p = [[0, 0.9, -1e-05]],
  shape = [0x560a03297190],
  uv = [[0.5, 5.30516e-06]],
  n = [[-0.0995087, 0.995037, -3.31679e-05]],
  sh_frame = Frame[
    s = [[0.995037, 0.0995087, -3.31696e-06]],
    t = [[2.12839e-13, -3.33333e-05, -1]],
    n = [[-0.0995087, 0.995037, -3.31679e-05]]
  ],
  dp_du = [[1, 0.1, -3.33333e-06]],
  dp_dv = [[0, -6.28319e-05, -1.88496]],
  dn_du = [[0, 0, 0]],
  dn_dv = [[0, 0, 0]],
  duv_dx = [[0, 0]],
  duv_dy = [[0, 0]],
  wi = [[0.0995087, -3.33333e-05, 0.995037]],
  prim_index = [2],
  instance = [0x0]
]

In [14]:
normal_theory = dr.normalize(mi.Vector3f(-0.1, 1, 0))
assert(dr.allclose(normal_theory, si.n, atol=1e-4))

In [15]:
uv_theory = mi.Vector2f(0.5, 0.0)
assert(dr.allclose(uv_theory, si.uv, atol=1e-4))

In [18]:
dp_du_theory = mi.Vector3f(1, 0.1, 0)
assert(dr.allclose(dp_du_theory, si.dp_du, atol=1e-4))

In [19]:
dp_dv_theory = mi.Vector3f(0, 0, -dr.two_pi * 0.3)
assert(dr.allclose(dp_dv_theory, si.dp_dv, atol=1e-4))